In [1]:
!pip install ratsnlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 KB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 KB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 582.5/582.5 KB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 35.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 517.2/517.2 KB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 KB 27.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 9.4 MB/s eta 0:00:00
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=4345a0385dcaecfa12c432b09202d5afa0c62f98f48e3d94f12ccc74d458927c
  Stored in directory: /root/.cache/pip/wheels/82/ab/9b/c1

In [2]:
from google.colab import drive
drive.mount('/gdrive', force_remount = True)

Mounted at /gdrive


In [3]:
from ratsnlp.nlpbook.classification import ClassificationDeployArguments
args = ClassificationDeployArguments(
    pretrained_model_name = "beomi/kcbert-base",
    downstream_model_dir = "/gdrive/My Drive/nlpbook/checkpoint-paircls",
    max_seq_length=64,
)

downstream_model_checkpoint_fpath: /gdrive/My Drive/nlpbook/checkpoint-paircls/epoch=1-val_loss=0.88.ckpt


In [4]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(
    args.pretrained_model_name,
    do_lower_case=False
)

Downloading:   0%|          | 0.00/250k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/619 [00:00<?, ?B/s]

In [5]:
import torch
fine_tuned_model_ckpt = torch.load(
    args.downstream_model_checkpoint_fpath,
    map_location=torch.device("cpu"),
)

In [6]:
from transformers import BertConfig
pretrained_model_config = BertConfig.from_pretrained(
    args.pretrained_model_name,
    num_labels = fine_tuned_model_ckpt['state_dict']['model.classifier.bias'].shape.numel(),
)

In [7]:
from transformers import BertForSequenceClassification
model = BertForSequenceClassification(pretrained_model_config)

In [8]:
model.load_state_dict ({k.replace("model.",""): v for k, v in fine_tuned_model_ckpt['state_dict'].items()})

<All keys matched successfully>

In [9]:
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30000, 768, padding_idx=0)
      (position_embeddings): Embedding(300, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [10]:
def inference_fn(person, system):
  inputs = tokenizer(
      [(person, system)],
      max_length = args.max_seq_length,
      padding = "max_length",
      truncation = True,
  )
  with torch.no_grad():
    outputs = model(**{k: torch.tensor(v) for k, v in inputs.items()})
    prob = outputs.logits.softmax(dim=1)
    anger_prob = round(prob[0][0].item(),2)
    unrest_prob = round(prob[0][1].item(),2)
    embarrassment_prob = round(prob[0][2].item(),2)
    sadness_prob = round(prob[0][3].item(),2)
    wound_prob = round(prob[0][4].item(),2)
    
    result = [anger_prob, unrest_prob, embarrassment_prob, sadness_prob, wound_prob]
    result_prob = max(result)

    if result_prob == anger_prob:
      final_emotion = 0
    elif result_prob == unrest_prob:
      final_emotion = 1
    elif result_prob == embarrassment_prob:
      final_emotion = 2
    elif result_prob == sadness_prob:
      final_emotion = 3
    else:
      final_emotion = 4


  return(final_emotion)
      
      #'person' : person,
      #"system" : system,
      #"anger_data" : f"분노 {anger_prob}",
      #"unrest_data" : f"불안 {unrest_prob}",
      #"embarrassment_data" : f"당황 {embarrassment_prob}",
      #"sadness_data" : f"슬픔 {sadness_prob}",
      #"wound_data" : f"상처 {wound_prob}",
      #"최종 감정" : final_emotion,

In [11]:
inference_fn("친구가 지금 이렇게 사는 게 내가 학창 시절에 노력하지 않아서라는 얘기를 해서 낙담했어.", "친구의 이야기가 당신을 좀 상처받게 했군요.")

3

In [12]:
import numpy as np
import pandas as pd
from tqdm import tqdm

In [13]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [14]:
eval_data = pd.read_csv("/content/drive/My Drive/NLP/testset.csv")
eval_data

,Unnamed: 0,person,GPT,emotion,emotion2
0,0,친구한테 말을 걸었는데 대꾸도 안 하고 다른 친구들이랑 나가더라.,"당신의 친구가 다른 사람들과 나가고 대답하지 않았다면, 그것은 그 친구가 현재 바쁘...",당황,2
1,1,중요한 보고서 제출일이 오늘까지였는데 이제 알게 돼서 너무 당황스러워.,"보고서 제출일이 이미 지났다면, 당황할 필요가 있지만 지금은 문제를 해결하기 위해 ...",당황,2
2,2,오늘 제출해야 하는 자료를 집에 두고 와서 나 스스로가 너무 한심하고 어떻게 해야 ...,침착하게 상황을 받아들이고 자신에게 너무 부담을 주지 마시고 가능한 한 조용하게 생...,당황,2
3,3,부장님께서 일주일 전에 부탁하신 일이 있었는데 까먹고 있다가 이제야 생각이 나서 너...,"이러한 상황은 일상적으로 발생하는 일이며, 누구나 실수를 할 수 있으니 그 실수를 ...",당황,2
4,4,팀장님이 나한테 부탁하신 서류를 잃어버렸어. 오늘 회의 자료라고 하셨는데 진짜 너무...,"당황하지 마시고, 가능한 빠르게 상황을 해결하는 것이 중요합니다. 이번 상황은 불가...",당황,2
...,...,...,...,...,...
145,145,친구들이랑 싸워서 너무 슬퍼.,"친구들과 싸우는 것은 어려운 상황이지만, 시간을 두고 마음을 가라앉히고, 서로의 생...",슬픔,3
146,146,어렸을 적 친구들과 점점 멀어지는 것 같아서 눈물이 나.,"어렸을 적 친구들과의 추억이 그리워서 슬퍼하는 것은 자연스러운 감정이지만, 시간이 ...",슬픔,3
147,147,요새 코로나 때문에 집에만 있어서 너무 우울해.,코로나로 인한 격리와 사회적 거리두기로 인해 많은 사람들이 우울감을 느끼고 있습니다.,슬픔,3
148,148,어제 내 딸이 사고를 당해서 다리를 다쳤어. 너무 슬퍼.,가족 중 한 명이 다친 것은 매우 힘든 상황입니다. 딸의 상태가 좋아지길 바랄 뿐입니다.,슬픔,3


In [15]:
eval_data = eval_data[['person', 'GPT', 'emotion2']]
eval_data.columns = ['person','system', 'origin']
eval_data

,person,system,origin
0,친구한테 말을 걸었는데 대꾸도 안 하고 다른 친구들이랑 나가더라.,"당신의 친구가 다른 사람들과 나가고 대답하지 않았다면, 그것은 그 친구가 현재 바쁘...",2
1,중요한 보고서 제출일이 오늘까지였는데 이제 알게 돼서 너무 당황스러워.,"보고서 제출일이 이미 지났다면, 당황할 필요가 있지만 지금은 문제를 해결하기 위해 ...",2
2,오늘 제출해야 하는 자료를 집에 두고 와서 나 스스로가 너무 한심하고 어떻게 해야 ...,침착하게 상황을 받아들이고 자신에게 너무 부담을 주지 마시고 가능한 한 조용하게 생...,2
3,부장님께서 일주일 전에 부탁하신 일이 있었는데 까먹고 있다가 이제야 생각이 나서 너...,"이러한 상황은 일상적으로 발생하는 일이며, 누구나 실수를 할 수 있으니 그 실수를 ...",2
4,팀장님이 나한테 부탁하신 서류를 잃어버렸어. 오늘 회의 자료라고 하셨는데 진짜 너무...,"당황하지 마시고, 가능한 빠르게 상황을 해결하는 것이 중요합니다. 이번 상황은 불가...",2
...,...,...,...
145,친구들이랑 싸워서 너무 슬퍼.,"친구들과 싸우는 것은 어려운 상황이지만, 시간을 두고 마음을 가라앉히고, 서로의 생...",3
146,어렸을 적 친구들과 점점 멀어지는 것 같아서 눈물이 나.,"어렸을 적 친구들과의 추억이 그리워서 슬퍼하는 것은 자연스러운 감정이지만, 시간이 ...",3
147,요새 코로나 때문에 집에만 있어서 너무 우울해.,코로나로 인한 격리와 사회적 거리두기로 인해 많은 사람들이 우울감을 느끼고 있습니다.,3
148,어제 내 딸이 사고를 당해서 다리를 다쳤어. 너무 슬퍼.,가족 중 한 명이 다친 것은 매우 힘든 상황입니다. 딸의 상태가 좋아지길 바랄 뿐입니다.,3


In [16]:
inference_fn(eval_data['person'].loc[0],eval_data['system'].loc[0])

1

In [17]:
result = []
for i in tqdm(range(eval_data.shape[0])):
  value = inference_fn(eval_data['person'].loc[i],eval_data['system'].loc[i])
  result.append(value)

100%|██████████| 150/150 [00:46<00:00,  3.20it/s]


In [18]:
eval_data['result'] = result

<ipython-input-18-952af87d708b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eval_data['result'] = result


In [25]:
origin = []
for i in tqdm(range(eval_data.shape[0])):
  org = eval_data['origin'].loc[i]
  origin.append(org)

100%|██████████| 150/150 [00:00<00:00, 18986.77it/s]


In [19]:
eval_data

,person,system,origin,result
0,친구한테 말을 걸었는데 대꾸도 안 하고 다른 친구들이랑 나가더라.,"당신의 친구가 다른 사람들과 나가고 대답하지 않았다면, 그것은 그 친구가 현재 바쁘...",2,1
1,중요한 보고서 제출일이 오늘까지였는데 이제 알게 돼서 너무 당황스러워.,"보고서 제출일이 이미 지났다면, 당황할 필요가 있지만 지금은 문제를 해결하기 위해 ...",2,2
2,오늘 제출해야 하는 자료를 집에 두고 와서 나 스스로가 너무 한심하고 어떻게 해야 ...,침착하게 상황을 받아들이고 자신에게 너무 부담을 주지 마시고 가능한 한 조용하게 생...,2,2
3,부장님께서 일주일 전에 부탁하신 일이 있었는데 까먹고 있다가 이제야 생각이 나서 너...,"이러한 상황은 일상적으로 발생하는 일이며, 누구나 실수를 할 수 있으니 그 실수를 ...",2,1
4,팀장님이 나한테 부탁하신 서류를 잃어버렸어. 오늘 회의 자료라고 하셨는데 진짜 너무...,"당황하지 마시고, 가능한 빠르게 상황을 해결하는 것이 중요합니다. 이번 상황은 불가...",2,1
...,...,...,...,...
145,친구들이랑 싸워서 너무 슬퍼.,"친구들과 싸우는 것은 어려운 상황이지만, 시간을 두고 마음을 가라앉히고, 서로의 생...",3,3
146,어렸을 적 친구들과 점점 멀어지는 것 같아서 눈물이 나.,"어렸을 적 친구들과의 추억이 그리워서 슬퍼하는 것은 자연스러운 감정이지만, 시간이 ...",3,3
147,요새 코로나 때문에 집에만 있어서 너무 우울해.,코로나로 인한 격리와 사회적 거리두기로 인해 많은 사람들이 우울감을 느끼고 있습니다.,3,3
148,어제 내 딸이 사고를 당해서 다리를 다쳤어. 너무 슬퍼.,가족 중 한 명이 다친 것은 매우 힘든 상황입니다. 딸의 상태가 좋아지길 바랄 뿐입니다.,3,3


In [20]:
score = []
for i in tqdm(range(eval_data.shape[0])):
  if eval_data['origin'].loc[i] == eval_data['result'].loc[i]:
      score.append(1)
  else:
      score.append(0)

eval_data['score'] = score

100%|██████████| 150/150 [00:00<00:00, 14257.29it/s]


In [21]:
eval_data

,person,system,origin,result,score
0,친구한테 말을 걸었는데 대꾸도 안 하고 다른 친구들이랑 나가더라.,"당신의 친구가 다른 사람들과 나가고 대답하지 않았다면, 그것은 그 친구가 현재 바쁘...",2,1,0
1,중요한 보고서 제출일이 오늘까지였는데 이제 알게 돼서 너무 당황스러워.,"보고서 제출일이 이미 지났다면, 당황할 필요가 있지만 지금은 문제를 해결하기 위해 ...",2,2,1
2,오늘 제출해야 하는 자료를 집에 두고 와서 나 스스로가 너무 한심하고 어떻게 해야 ...,침착하게 상황을 받아들이고 자신에게 너무 부담을 주지 마시고 가능한 한 조용하게 생...,2,2,1
3,부장님께서 일주일 전에 부탁하신 일이 있었는데 까먹고 있다가 이제야 생각이 나서 너...,"이러한 상황은 일상적으로 발생하는 일이며, 누구나 실수를 할 수 있으니 그 실수를 ...",2,1,0
4,팀장님이 나한테 부탁하신 서류를 잃어버렸어. 오늘 회의 자료라고 하셨는데 진짜 너무...,"당황하지 마시고, 가능한 빠르게 상황을 해결하는 것이 중요합니다. 이번 상황은 불가...",2,1,0
...,...,...,...,...,...
145,친구들이랑 싸워서 너무 슬퍼.,"친구들과 싸우는 것은 어려운 상황이지만, 시간을 두고 마음을 가라앉히고, 서로의 생...",3,3,1
146,어렸을 적 친구들과 점점 멀어지는 것 같아서 눈물이 나.,"어렸을 적 친구들과의 추억이 그리워서 슬퍼하는 것은 자연스러운 감정이지만, 시간이 ...",3,3,1
147,요새 코로나 때문에 집에만 있어서 너무 우울해.,코로나로 인한 격리와 사회적 거리두기로 인해 많은 사람들이 우울감을 느끼고 있습니다.,3,3,1
148,어제 내 딸이 사고를 당해서 다리를 다쳤어. 너무 슬퍼.,가족 중 한 명이 다친 것은 매우 힘든 상황입니다. 딸의 상태가 좋아지길 바랄 뿐입니다.,3,3,1


In [22]:
sum(eval_data['score'])

120

In [23]:
sum(eval_data['score'])/eval_data.shape[0]*100

80.0

In [24]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [26]:
target_names = ['분노', '불안', '당황', '슬픔', '상처']

print(confusion_matrix(origin, result))
print(classification_report(origin, result, target_names=target_names))

[[23  6  1  0  0]
 [ 0 29  0  0  1]
 [ 1 10 18  0  1]
 [ 0  0  0 28  2]
 [ 0  2  1  5 22]]
              precision    recall  f1-score   support

          분노       0.96      0.77      0.85        30
          불안       0.62      0.97      0.75        30
          당황       0.90      0.60      0.72        30
          슬픔       0.85      0.93      0.89        30
          상처       0.85      0.73      0.79        30

    accuracy                           0.80       150
   macro avg       0.83      0.80      0.80       150
weighted avg       0.83      0.80      0.80       150

